In [8]:
import os
import chromadb
from langchain.vectorstores import Chroma
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.retrievers.merger_retriever import MergerRetriever

In [3]:
model_name = "BAAI/bge-large-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
print("Embedding Model Loaded..........")

Embedding Model Loaded..........


In [4]:
load_un_sdg_store = Chroma(persist_directory="store/un_sdg_chroma_cosine", embedding_function=hf)
print("First Vector Store Loaded.........")

First Vector Store Loaded.........


In [5]:
load_paris_agreement_store = Chroma(persist_directory="store/paris_chroma_cosine", embedding_function=hf)
print("Second Vector Store Loaded........")

Second Vector Store Loaded........


In [6]:
retriever_un_sdg = load_un_sdg_store.as_retriever(search_type = "similarity", search_kwargs = {"k":3, "include_metadata": True})

retriever_paris_agreement = load_paris_agreement_store.as_retriever(search_type = "similarity", search_kwargs = {"k":3, "include_metadata": True})

In [9]:
lotr = MergerRetriever(retrievers=[retriever_un_sdg, retriever_paris_agreement])

In [10]:
for chunks in lotr.get_relevant_documents("Is there any framework available to tackle the climate change?"):
    print(chunks.page_content)

resilience+ and+ adaptive+ capacity+ to+ climateJrelated+ hazards+ and+ natural+disasters+in+all+countries++13.2+Integrate+climate+change+measures+into+national+policies,+strategies+and+planning++13.3+Improve+education,+awarenessJraising+and+human+and+institutional+capacity+on+climate+change+mitigation,+adaptation,+impact+reduction+and+early+warning+13.a+Implement+the+commitment+undertaken+by+developedJcountry+parties+to+the+United+Nations+Framework+Convention+on+Climate+Change+to+a+goal+of+mobilizing+jointly+$100+billion+annually+by+2020+from+all+sources+to+address+the+needs+of+developing+countries+in+the+context+of+meaningful+mitigation+actions+and+transparency+on+implementation+and+fully+operationalize+the+Green+Climate+Fund+through+its+capitalization+as+soon+as+possible+++*+Acknowledging+that+the+United+Nations+Framework+Convention+on+Climate+Change+is+the+primary+international,+intergovernmental+forum+for+negotiating+the+global+response+to+climate+change.+
finance should  represen

In [11]:
query = "Is there any framework available to tackle the climate change?"

In [12]:
docs = lotr.get_relevant_documents(query)
docs

[Document(page_content='resilience+ and+ adaptive+ capacity+ to+ climateJrelated+ hazards+ and+ natural+disasters+in+all+countries++13.2+Integrate+climate+change+measures+into+national+policies,+strategies+and+planning++13.3+Improve+education,+awarenessJraising+and+human+and+institutional+capacity+on+climate+change+mitigation,+adaptation,+impact+reduction+and+early+warning+13.a+Implement+the+commitment+undertaken+by+developedJcountry+parties+to+the+United+Nations+Framework+Convention+on+Climate+Change+to+a+goal+of+mobilizing+jointly+$100+billion+annually+by+2020+from+all+sources+to+address+the+needs+of+developing+countries+in+the+context+of+meaningful+mitigation+actions+and+transparency+on+implementation+and+fully+operationalize+the+Green+Climate+Fund+through+its+capitalization+as+soon+as+possible+++*+Acknowledging+that+the+United+Nations+Framework+Convention+on+Climate+Change+is+the+primary+international,+intergovernmental+forum+for+negotiating+the+global+response+to+climate+change.+'

## Reordered Docs (Tackling Lost in the Middle)

In [13]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(page_content='finance should  represent a progression beyond previous efforts. \n4. The provision of scaled-up financial resources should aim to achieve a \nbalance between adaptation and mitigation, taking into account country-driven \nstrategies, and the priorities and needs of developing country Parties, especially \nthose that are particularly vulnerable to the adverse effects of climate change and \nhave significant capacity constraints, such as the least developed countries and \nsmall island developing States, considering the need for public and grant-based \nresources for adaptation. \n5. Developed country Parties shall biennially communicate indicative \nquantitative and qualitative information related to paragraphs 1 and 3 of this \nArticle, as applicable, including, as available, projected levels of public financial \nresources to be provided to developing country Parties. Other Parties providing \nresources are encouraged to communicate biennially such information

## 